# Amazon Reviews dataset, collected in 2023

- Official link: https://amazon-reviews-2023.github.io/main.html
- Category: Electronics
- Users: 18.3M
- Items: 1.6M
- Ratings: 43.9M
- R_Token: 2.7B
- M_Token: 1.7B
- review: https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/review_categories/Electronics.jsonl.gz
- meta: https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/meta_categories/meta_Electronics.jsonl.gz 

### Dependencies

In [ ]:
import json
import pandas as pd

## Sampling
---

### 1. Filter Items that have been observed for the first time in year 2022 or later.

In [ ]:
def filter_data(data: dict) -> dict:
    '''Return boolean values, where True means the data is not valid'''
    filter = False
    if int(data['details']['Date First Available'][-4:]) < 2022:
        filter = True

    return filter

In [ ]:
with open("../data/raw/meta_Electronics.jsonl", 'r') as fp:
    with open("../data/meta_Electronics_2022_2023.jsonl", 'a', encoding='utf-8') as fp_out:
        with open("../data/drop/meta_Electronics_2022_2023_no_date.jsonl", 'a', encoding='utf-8') as fp_out_no_date:
            i = 0
            for line in fp:
                data = json.loads(line.strip())
                try:
                    filter = filter_data(data)
                    if not filter:
                        json.dump(data, fp_out)
                        fp_out.write('\n')
                        fp_out.flush()
                except:
                    json.dump(data, fp_out_no_date)
                    fp_out_no_date.write('\n')
                    fp_out_no_date.flush()
                i += 1
                if i % 10000 == 0:
                    print(f"Processed {i/1000} k lines")

### 2. Filter Items, where "main category" is not empty

In [ ]:
def filter_category(data: dict) -> dict:
    '''Return boolean values, where True means the data is not valid'''
    filter = False
    if data['main_category'] == None:
        filter = True

    return filter

In [ ]:
with open("../data/meta_Electronics_2022_2023.jsonl", 'r') as fp:
    with open("../data/meta_Electronics_2022_2023_with_category.jsonl", 'a', encoding='utf-8') as fp_out:
        with open("../data/drop/meta_Electronics_2022_2023_no_category.jsonl", 'a', encoding='utf-8') as fp_out_no_category:
            for line in fp:
                data = json.loads(line.strip())
                if not filter_category(data):
                    json.dump(data, fp_out)
                    fp_out.write('\n')
                    fp_out.flush()
                else:
                    json.dump(data, fp_out_no_category)
                    fp_out_no_category.write('\n')
                    fp_out_no_category.flush()

Explore dsitribution by categories

In [ ]:
df = pd.read_json("../data/meta_Electronics_2022_2023_with_category.jsonl", lines=True)
print(df.shape)

In [ ]:
len(df['main_category'].unique()), df['main_category'].unique()

In [ ]:
df['main_category'].value_counts().plot(kind='bar')

### Filter out items that have at least 100 ratings

In [ ]:
df_ratings_100 = df[df['rating_number'] > 100]

Explore distribution of ratings

In [ ]:
df_ratings_100['average_rating'].plot(kind='hist', bins=50, range=(0, 5))

### Sample 1000 items

In [ ]:
df_sample_1000 = df_ratings_100.sample(n=1000, random_state=20)

In [ ]:
df_sample_1000['average_rating'].plot(kind='hist', bins=50, range=(0, 5))

In [ ]:
df_sample_1000['price'].plot(kind='hist', bins=100, range=(0, 500))

In [ ]:
df_ratings_100.to_json("../data/meta_Electronics_2022_2023_with_category_ratings_100.jsonl", orient='records', lines=True)

In [ ]:
df_sample_1000.to_json("../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", orient='records', lines=True)

## Extract ratings that match sampled data.

In [ ]:
df_ratings_100 = pd.read_json("../data/meta_Electronics_2022_2023_with_category_ratings_100.jsonl", lines=True)
df_sample_1000 = pd.read_json("../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [ ]:
# Gabriels suggestion
id_set = set(df_ratings_100['parent_asin'].values)

with open("../data/raw/Electronics.jsonl", "r") as fp, \
     open("../data/Electronics_2022_2023_with_category_ratings_100.jsonl", "a", encoding="utf-8") as fp_out:

    for i, line in enumerate(fp, 1):
        data = json.loads(line.strip())
        if data.get('parent_asin') in id_set:
            json.dump(data, fp_out)
            fp_out.write("\n")

        if i % 100000 == 0:
            print(f"Processed {i/1000} k lines")

In [ ]:
# with open("../data/Electronics.jsonl", 'r') as fp:
#     with open("../data/Electornics_2022_2023_with_category_ratings_100.jsonl", 'a') as fp_out:
#         id_list = df_ratings_100['parent_asin'].values
#         i = 0
#         for line in fp:
#             data = json.loads(line.strip())
#             if data['parent_asin'] in id_list:
#                 json.dump(data, fp_out)
#                 fp_out.write('\n')
#                 fp_out.flush()
#             i += 1
#             if i % 100000 == 0:
#                 print(f"Processed {i} lines")

In [ ]:
with open("../data/Electronics_2022_2023_with_category_ratings_100.jsonl", 'r') as fp:
    with open("../data/Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", 'a') as fp_out:
        id_list = df_sample_1000['parent_asin'].values
        i = 0
        for line in fp:
            data = json.loads(line.strip())
            if data['parent_asin'] in id_list:
                json.dump(data, fp_out)
                fp_out.write('\n')
                fp_out.flush()
            i += 1
            if i % 100000 == 0:
                print(f"Processed {i/1000} k lines")

---

## Sneak peak

In [ ]:
import pandas as pd
import json

def read_jsonl_to_dataframe(file_path, max_rows=None):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if max_rows and i >= max_rows:
                break
            data.append(json.loads(line.strip()))
    return pd.DataFrame(data)

In [ ]:
file_path_ratings = "../data/Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl"
file_path_meta = "../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl"

df_ratings = read_jsonl_to_dataframe(file_path_ratings)
df_meta = read_jsonl_to_dataframe(file_path_meta)


### Meta data

In [ ]:
print(df_meta.shape)
print(df_meta.columns)
display(df_meta.head())

In [ ]:
# Count NAs
# 170 prices are missing
# features & descriptions mean the same thing
print(df_meta.isna().sum())
display(df_meta[df_meta['price'].isna()].head(10))

In [ ]:
# Cardinality
def calculate_cardinality(series):
    try:
        return series.nunique()
    except TypeError:
        # For unhashable types like lists, convert to strings first
        return series.astype(str).nunique()

# Apply
cardinality = df_meta.apply(calculate_cardinality)
print(cardinality)

In [ ]:
# Category summary
category_counts = df_meta.groupby('main_category').size().sort_values(ascending=False)
total_count = category_counts.sum()

# Summary df
result_df = pd.DataFrame({
    'main_category': category_counts.index,
    'count': category_counts.values,
    'percentage': (category_counts.values / total_count * 100).round(2)
})

# Example titles
example_titles = []
for category in result_df['main_category']:
    titles = df_meta[df_meta['main_category'] == category]['title'].head(5).tolist()
    example_titles.append(' | '.join(titles))
result_df['example_titles'] = example_titles

display(result_df)

In [ ]:
# Weird categories
print("Health & Personal Care")
for title in df_meta[df_meta['main_category'] == 'Health & Personal Care'].title:
    print(f"- {title}")

print("Sports & Outdoors")
for title in df_meta[df_meta['main_category'] == 'Sports & Outdoors'].title:
    print(f"- {title}")

print("Tools & Home Improvement	") # mostly cable management and organization accessories
for title in df_meta[df_meta['main_category'] == 'Tools & Home Improvement'].title:
    print(f"- {title}")

print("Amazon Home") # mostly stickers
for title in df_meta[df_meta['main_category'] == 'Amazon Home'].title:
    print(f"- {title}")

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
# Price distribution
ax1.hist(df_meta['price'].dropna(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax1.set_title('Price')
ax1.grid(True, alpha=0.3)

# Average rating distribution
ax2.hist(df_meta['average_rating'].dropna(), bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
ax2.set_title('Average rating')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Some statistics
print("Price Statistics:")
print(df_meta[['price', 'average_rating']].describe())

In [ ]:
top3_categories = df_meta['main_category'].value_counts().head(3).index

for category in top3_categories:
    print("-" * 100)
    print(f"Category: {category}")
    
    df_category = df_meta[df_meta['main_category'] == category]
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
    
    # Price distribution
    ax1.hist(df_category['price'].dropna(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.set_title('Price')
    ax1.grid(True, alpha=0.3)

    # Average rating distribution
    ax2.hist(df_category['average_rating'].dropna(), bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
    ax2.set_title('Average rating')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Some statistics
    print("Price Statistics:")
    print(df_category[['price', 'average_rating']].describe())
    print(f"Most expensive product: {df_category['price'].max()}: {df_category[df_category['price'] == df_category['price'].max()].title.values[0]}")

### Ratings

In [ ]:
print(df_ratings.shape)
print(df_ratings.columns)
display(df_ratings.head())

In [ ]:
# Count NAs
print(df_ratings.isna().sum())

In [ ]:
# Unique users
print(df_ratings['user_id'].nunique())

# Unique products
print(df_ratings['asin'].nunique())

# Unique products
print(df_ratings['parent_asin'].nunique())


In [ ]:
# Count records per parent_asin
print("Ratings per item (parent)")
print(df_ratings['parent_asin'].value_counts())

# Count items with less then 10 ratings
print(f"Items with less then 10 ratings: {df_ratings['parent_asin'].value_counts()[df_ratings['parent_asin'].value_counts() < 10].shape[0]}")	

In [ ]:
# Counts of each parent_asin
asin_counts = df_ratings['parent_asin'].value_counts()

# Intervals 
bins = [0, 9, 49, 99, 499, 999, 1999, 2999, 3999, 4999, float('inf')]
labels = ['1-9', '10-49', '50-99', '100-499', '500-999', '1000-1999', '2000-2999', '3000-3999', '4000-4999', '5000+']
interval_counts = pd.cut(asin_counts, bins=bins, labels=labels, include_lowest=True).value_counts().sort_index()

# To df
intervals_df = pd.DataFrame({
    'interval_of_number_of_ratings': interval_counts.index,
    'number_of_items': interval_counts.values,
    'percentage_of_items': (interval_counts.values / interval_counts.sum() * 100).round(2)
})

display(intervals_df)

In [ ]:
# Filter items with most ratings
df_ratings_filtered = df_ratings[df_ratings['parent_asin'].isin(df_ratings['parent_asin'].value_counts()[df_ratings['parent_asin'].value_counts() > 3000].index)]

print('Items with most ratings (3000+)')
# Count unique users
print(f"- Number of records: {df_ratings_filtered.shape[0]}")
print(f"- Number of users: {df_ratings_filtered['user_id'].nunique()}")
print(f"- Number of items (parent): {df_ratings_filtered['parent_asin'].nunique()}")
print(f"- Number of ratings: {df_ratings_filtered['asin'].nunique()}")

print("Ratings per item (parent) in most rated items (3000+)")
# Ratings per item
print(df_ratings_filtered['parent_asin'].value_counts())

In [ ]:
# Most rated item
display(df_meta[df_meta['parent_asin']=='B0C7Q3X76Q'])
print(df_ratings[df_ratings['parent_asin']=='B0C7Q3X76Q'].shape)
display(df_ratings[df_ratings['parent_asin']=='B0C7Q3X76Q'])